For this demonstration of energy filtered transmission electron microscopy (EFTEM) we will be simulating a SrTiO3 CeO2 interface downloaded from https://www.materialscloud.org/explore/stoceriaitf/grid/calculations. We first import the multislice library, read in and plot the crystal and set up some basic parameters for our simulation. This calculation took approximately 7 mins 47 on my 2GB NVIDIA GTX 750Ti.

In [ ]:
# Import multislice library, numpy, torch and matplotlib for plotting
import pyms
import numpy as np
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Probe accelerating voltage in eV
eV = 3e5

# No objective aperture
app = None

# Simulation pixel grid shape
gridshape = [512,512]

thicknesses = 100

# Make SrTiO3/CeO2 interface structure
# Get crystal
crystal = pyms.structure.fromfile(
    "Structures/SrTiO3_CeO2_interface.xyz", atomic_coordinates="cartesian"
)

# tiling to make a square object
tiling = [1,7]

# A few maniupulations to remove vaccuum at edges of the input 
# cell and create a psuedo-periodic structure for multislice
crystal = crystal.resize([[0.1, 0.76]], axis=0)
from copy import deepcopy
other = deepcopy(crystal).resize([[0.017, 0.99]], axis=0)
other.reflect([0])
crystal = crystal.concatenate(other, axis=0)

# Subslicing of crystal for multislice
subslices = [0.33, 0.66, 1.0]

# Output structure for examination in Vesta to check
# crystal.output_vesta_xtl("manipulated.xtl")

# Grid size in Angstrom
gridsize = np.asarray(tiling)*crystal.unitcell[:2]

#Defocii for an image series
defocii = np.arange(-300,301,150)

# Choose device (CPU or GPU) to run calculations on
device = torch.device('cuda:0')
# device = torch.device('cpu')

# Target the oxygen K edge
Ztarget = 8
# principal and orbital angular momentum quantum numbers for the 
# bound state of the oxygen 1s orbital
n=1
ell=0
# Energy above ionization threshhold
epsilon = 1

# Choose between 

EFTEM_images = pyms.EFTEM(
    crystal,
    gridshape,
    eV,
    app,
    thicknesses,
    Ztarget,
    n,
    ell,
    epsilon,
    df=defocii,
    subslices=subslices,
    device_type=None,
    tiling=tiling,
    nT=5,
    showProgress = 'notebook',
)

#Plot image series
N = int(np.round(np.sqrt(EFTEM_images.shape[0])))
fig,axes = plt.subplots(nrows= EFTEM_images.shape[0],figsize=(4,16))
for ax,img,df in zip(axes.ravel(),EFTEM_images,defocii):

    ax.imshow(img,cmap = plt.get_cmap('gray'))
    ax.set_axis_off()
    ax.set_title('Defocus {0:5.0f} \AA'.format(df),usetex=True)
